<a href="https://colab.research.google.com/github/esuda/mineiracao_dados_complexos/blob/master/Big%20Data/Big_Data_010_Trabalho_3_B3_Noriaki_Suda_Wakabayashi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrantes do Grupo
* Daniel Noriaki Kurosawa
* Eric Uyemura Suda
* Fernando Shigeru Wakabayashi

# Dados de Entrada

*   [Pacote de dados](https://tinyurl.com/bd009-b3)
  *   Selecione "Adicionar ao Drive"
*   [Descrição dos dados](https://drive.google.com/file/d/1wu1yfR1A0A0Tl8Jl1ZNNuowr88YEFtXS/view?usp=sharing)






In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Instalação de pacotes

In [ ]:
!wget -q https://downloads.apache.org/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz
!pip install findspark pyspark 

     |████████████████████████████████| 281.3 MB 8.0 kB/s 
     |████████████████████████████████| 198 kB 46.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=1769f8cbcded5442378e8970d601a81a9915d1f7509a5bb5d80171df26ecb1ae
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


# Preparação do ambiente

In [ ]:
%env PYTHONHASHSEED=1234
%env JAVA_HOME=/usr/lib/jvm/default-java
%env SPARK_HOME=/content/spark-3.2.0-bin-hadoop3.2

env: PYTHONHASHSEED=1234
env: JAVA_HOME=/usr/lib/jvm/default-java
env: SPARK_HOME=/content/spark-3.2.0-bin-hadoop3.2


In [ ]:
import findspark
findspark.init("/content/spark-3.2.0-bin-hadoop3.2")

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import Window


from datetime import datetime

appName = 'Big Data'
master = 'local'

spark = SparkSession.builder     \
    .master(master) \
    .appName(appName) \
    .getOrCreate()

spark.sparkContext.setLogLevel("WARN")

# Leitura dos dados e criação de dataframes

In [ ]:
def read_line(line) :
  tipo = line[0:2]
  if tipo == '01' :
    yield Row(tipreg=int(line[0:2]),
               data=datetime.strptime(line[2:10], '%Y%m%d'),
               codbdi=int(line[10:12]),
               codneg=line[12:24].strip(),
               tpmerc=int(line[24:27]),
               especi=line[39:49].replace('*', '').replace(' ',''),
               preult=float(line[108:121])/100)




In [ ]:
# Modifique esta linha para incluir dados de outros anos na solução final

input_data = spark.sparkContext.textFile('/content/drive/My Drive/b3/COTAHIST*')

cotacoes = input_data.flatMap(read_line).toDF()
cotacoes.printSchema()
cotacoes.show()


root
 |-- tipreg: long (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- codbdi: long (nullable = true)
 |-- codneg: string (nullable = true)
 |-- tpmerc: long (nullable = true)
 |-- especi: string (nullable = true)
 |-- preult: double (nullable = true)

+------+-------------------+------+------+------+------+------+
|tipreg|               data|codbdi|codneg|tpmerc|especi|preult|
+------+-------------------+------+------+------+------+------+
|     1|1995-01-02 00:00:00|     2| ACE 3|    10| ONINT|  63.0|
|     1|1995-01-02 00:00:00|     2| ACE 4|    10| PNINT|  72.0|
|     1|1995-01-02 00:00:00|     2| ALP 4|    10|    PN| 160.0|
|     1|1995-01-02 00:00:00|     2| ROS 4|    10|    PN|  1.85|
|     1|1995-01-02 00:00:00|     2| BAS 4|    10| PNI94|  48.0|
|     1|1995-01-02 00:00:00|     2| AQT 4|    10|    PN|  0.78|
|     1|1995-01-02 00:00:00|     2| ARC 6|    10|   PNB|2260.0|
|     1|1995-01-02 00:00:00|     2| ATE 4|    10|    PN|   8.9|
|     1|1995-01-02 00:00:00| 

# Implementação

## Retirando os registros com NA

In [ ]:
cotacoes = cotacoes.na.drop()

## Primeiro Filtro
* Tipo Registro = 1 (Cotacao Historica)
* Codigo BDI = 02 (Lote Padrao)
* Tipo de Mercado = 010 (Mercado a Vista)

In [ ]:
cotacoes.printSchema()

root
 |-- tipreg: long (nullable = true)
 |-- data: timestamp (nullable = true)
 |-- codbdi: long (nullable = true)
 |-- codneg: string (nullable = true)
 |-- tpmerc: long (nullable = true)
 |-- especi: string (nullable = true)
 |-- preult: double (nullable = true)



In [ ]:
cotacoes_aux_01 = cotacoes.filter(cotacoes.tipreg == 1)\
                          .filter(cotacoes.codbdi == 2)\
                          .filter(cotacoes.tpmerc == 10)

In [ ]:
cotacoes_aux_01.show()

+------+-------------------+------+------+------+------+-------+
|tipreg|               data|codbdi|codneg|tpmerc|especi| preult|
+------+-------------------+------+------+------+------+-------+
|     1|2020-01-02 00:00:00|     2| AALR3|    10|  ONNM|   19.0|
|     1|2020-01-02 00:00:00|     2|AAPL34|    10|   DRN| 121.34|
|     1|2020-01-02 00:00:00|     2| ABCB4|    10|PNEJN2|   20.3|
|     1|2020-01-02 00:00:00|     2| ABEV3|    10|  ONEJ|   19.2|
|     1|2020-01-02 00:00:00|     2| ADHM3|    10|    ON|   2.48|
|     1|2020-01-02 00:00:00|     2| AFLT3|    10|    ON|  11.23|
|     1|2020-01-02 00:00:00|     2| AGRO3|    10|  ONNM|  19.25|
|     1|2020-01-02 00:00:00|     2| ALPA3|    10|  ONN1|   26.8|
|     1|2020-01-02 00:00:00|     2| ALPA4|    10|  PNN1|  32.58|
|     1|2020-01-02 00:00:00|     2| ALSO3|    10|  ONNM|  50.01|
|     1|2020-01-02 00:00:00|     2| ALUP3|    10|  ONN2|  11.66|
|     1|2020-01-02 00:00:00|     2| ALUP4|    10|  PNN2|   8.42|
|     1|2020-01-02 00:00:

## Segundo Filtro
* Apenas acoes ordinarias normativas ou ordinarias normativas novo mercado (retirar acoes que tenham especificacoes mistas);
* Desconsiderar acoes com especificacao igual " " ou "*"

In [ ]:
cotacoes_nao_on = cotacoes_aux_01.filter((col("especi") != "ON") & (col("especi") != "ONNM"))\
                          .select(col("codneg").alias("codneg_nao_on"),
                                  expr("1").alias("flag_nao_on"))\
                          .distinct()

In [ ]:
cotacoes_nao_on.show()

+-------------+-----------+
|codneg_nao_on|flag_nao_on|
+-------------+-----------+
|       A1MT34|          1|
|       G1RM34|          1|
|        BSLI3|          1|
|        EUCA3|          1|
|        FRAS3|          1|
|       I1VZ34|          1|
|       S1NP34|          1|
|        CRPG5|          1|
|        BRSR5|          1|
|       M1CB34|          1|
|       N1LS34|          1|
|       P1GR34|          1|
|       Z1IO34|          1|
|       Z1BH34|          1|
|        ENGI4|          1|
|       SIMN34|          1|
|       TMOS34|          1|
|       UPAC34|          1|
|       T1SO34|          1|
|       A1FL34|          1|
+-------------+-----------+
only showing top 20 rows



In [ ]:
cotacoes_aux_02 = cotacoes_aux_01.join(cotacoes_nao_on, 
                                       on=[cotacoes_aux_01.codneg == cotacoes_nao_on.codneg_nao_on], 
                                       how="left")

In [ ]:
cotacoes_aux_02.filter("flag_nao_on = 1 and (especi = 'ON' or especi = 'ONNM')").select("codneg").distinct().show()

+------+
|codneg|
+------+
| VAG 3|
| LAM 3|
| MSA 3|
| FCA 3|
| CRU 3|
| ELC 3|
| BAS 3|
| IAP 3|
| BEP 3|
| ITS 3|
| CMI 3|
| TER 3|
| LOB 3|
| IBA 3|
| VSM 3|
| BMK 3|
| NOR 3|
| MAH 3|
| UCA 3|
| MLF 3|
+------+
only showing top 20 rows



In [ ]:
cotacoes_aux_02.filter("codneg = 'B3SA3'").select("especi").distinct().show()

+-------+
| especi|
+-------+
| ONEJNM|
|   ONNM|
|ONEDJNM|
+-------+



In [ ]:
cotacoes_invalidas = cotacoes_aux_02.filter(col("especi").isin("", " ", "*"))\
                          .select(col("codneg").alias("codneg_invalido"),
                                  expr("1").alias("flag_invalido"))\
                          .distinct()

In [ ]:
# Para 2012 nao ha acoes com especificacao igaul a " " ou "*"
# mas manteremos o cruzamento para os outros anos
cotacoes_invalidas.show()

+---------------+-------------+
|codneg_invalido|flag_invalido|
+---------------+-------------+
+---------------+-------------+



In [ ]:
cotacoes_aux_03 = cotacoes_aux_02.join(cotacoes_invalidas, 
                                       on=[cotacoes_aux_02.codneg == cotacoes_invalidas.codneg_invalido], 
                                       how="left")

In [ ]:
cotacoes_aux_03.show()

+------+-------------------+------+------+------+------+-------+-------------+-----------+---------------+-------------+
|tipreg|               data|codbdi|codneg|tpmerc|especi| preult|codneg_nao_on|flag_nao_on|codneg_invalido|flag_invalido|
+------+-------------------+------+------+------+------+-------+-------------+-----------+---------------+-------------+
|     1|2020-01-02 00:00:00|     2| AZUL4|    10|  PNN2|   58.8|        AZUL4|          1|           null|         null|
|     1|2020-01-02 00:00:00|     2| ARZZ3|    10|  ONNM|  63.91|        ARZZ3|          1|           null|         null|
|     1|2020-01-02 00:00:00|     2| AGRO3|    10|  ONNM|  19.25|        AGRO3|          1|           null|         null|
|     1|2020-01-02 00:00:00|     2| ATOM3|    10|    ON|   1.72|         null|       null|           null|         null|
|     1|2020-01-02 00:00:00|     2| ADHM3|    10|    ON|   2.48|        ADHM3|          1|           null|         null|
|     1|2020-01-02 00:00:00|    

### Filtrando as flags de nao ON ONNM e campo invalido

In [ ]:
cotacoes_aux_04 = cotacoes_aux_03.filter("flag_nao_on is null and flag_invalido is null")\
                        .drop("codneg_nao_on", "flag_nao_on", "codneg_invalido", "flag_invalido")

In [ ]:
cotacoes_aux_04.select("codneg").distinct().show()

+------+
|codneg|
+------+
| LIGT3|
| MILS3|
| PRIO3|
| EKTR3|
| RSID3|
| VULC3|
| PTNT3|
| BRGE3|
| LPSB3|
| NAFG3|
| ENEV3|
| TESA3|
| VLID3|
| EMBR3|
| MNPR3|
| BOBR3|
| PLAS3|
| CRPG3|
| ATOM3|
| BRFS3|
+------+
only showing top 20 rows



## Calculo da volatilidade dos papeis

In [ ]:
janela_cotacao = Window.partitionBy('codneg').orderBy("data")

cotacoes_aux_04 = cotacoes_aux_04.withColumn("preult_ant", lag("preult", 1).over(janela_cotacao))\
                                  .withColumn("dif_rel", (col("preult")/col("preult_ant"))-1)\
                                  .filter("dif_rel is not null")


In [ ]:
cotacoes.filter("codneg = 'AFLU3'").sort("data", "dif_rel").show()

+------+-------------------+------+------+------+------+------+
|tipreg|               data|codbdi|codneg|tpmerc|especi|preult|
+------+-------------------+------+------+------+------+------+
|     1|2012-01-19 00:00:00|     2| AFLU3|    10|    ON|  4.93|
|     1|2012-02-01 00:00:00|     2| AFLU3|    10|    ON|   5.0|
|     1|2012-02-13 00:00:00|     2| AFLU3|    10|    ON|  5.27|
|     1|2012-02-14 00:00:00|     2| AFLU3|    10|    ON|   5.0|
|     1|2012-02-17 00:00:00|     2| AFLU3|    10|    ON|   5.0|
|     1|2012-03-01 00:00:00|     2| AFLU3|    10|    ON|   5.0|
|     1|2012-03-13 00:00:00|     2| AFLU3|    10|    ON|   5.2|
|     1|2012-03-14 00:00:00|     2| AFLU3|    10|    ON|   5.2|
|     1|2012-05-09 00:00:00|     2| AFLU3|    10|    ON|  4.83|
|     1|2012-06-11 00:00:00|     2| AFLU3|    10|    ON|  4.61|
|     1|2012-06-29 00:00:00|     2| AFLU3|    10|    ON|   6.1|
|     1|2012-08-06 00:00:00|     2| AFLU3|    10|    ON|  6.15|
|     1|2012-08-16 00:00:00|     2| AFLU

In [ ]:
cotacoes_var = cotacoes_aux_04.select("codneg", "dif_rel")\
                              .groupby("codneg")\
                              .agg(stddev("dif_rel"))

In [ ]:
cotacoes_var.show(cotacoes.count())

+------+--------------------+
|codneg|stddev_samp(dif_rel)|
+------+--------------------+
| AERI3|0.035309326149144345|
| AESL3|  120.94500665036315|
| AGR 3| 0.40176101334529724|
| AHE 3| 0.11296519899091297|
| ALPK3| 0.01787184515479393|
| ALSO3| 0.04005735515563661|
| AMBP3| 0.02123610033741466|
| AMCE3| 0.43256056304718327|
| AMPI3|  0.3398704488476588|
| AORE3| 0.10892028606398707|
| ARA 3|  0.5878275899627735|
| ARL 3|   0.490448532142979|
| ARP 3|  0.7777680333781121|
| ARTE3|  28.619386213880993|
| ASS 3|  0.7163634496417357|
| AVLL3|0.027641514583680433|
| BAN 3|  0.6271729948365894|
| BAQU3|  0.0778374811346549|
| BAR 3| 0.08700979781061385|
| BBVT3| 0.08361667522867804|
| BDE 3| 0.16606898780262738|
| BDEP3| 0.47150271983819436|
| BDL 3| 0.12515991890869743|
| BEBA3|                null|
| BEE 3| 0.22706295322298525|
| BEG 3| 0.22655259190481114|
| BER 3|  1.0695712629770202|
| BES 3| 0.05739273180161253|
| BEX 3|                null|
| BHEQ3|  2.0042586236494637|
| BIO 3|  

In [ ]:
cotacoes_aux_04.cache().count()

14837

In [ ]:
cotacoes_aux_04.filter("codneg in ('SPG 3','ENL 5','FCH 3')").sort("codneg", "data").show()

+------+-------------------+------+------+------+------+------+----------+------------------+
|tipreg|               data|codbdi|codneg|tpmerc|especi|preult|preult_ant|           dif_rel|
+------+-------------------+------+------+------+------+------+----------+------------------+
|     1|1997-02-07 00:00:00|     2| ENL 5|    10|    ON|   1.0|       1.0|               0.0|
|     1|1995-05-11 00:00:00|     2| FCH 3|    10|    ON|197.99|     130.0|0.5230000000000001|
|     1|1996-12-04 00:00:00|     2| SPG 3|    10|    ON|  3.51|      2.01|0.7462686567164181|
+------+-------------------+------+------+------+------+------+----------+------------------+

